In [44]:
"""ライブラリ"""
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from statistics import mean, variance
from scipy import stats
from scipy.stats import norm
from collections import Counter
import copy

In [45]:
"""データ生成関数"""
def generate_data(directory, data_n, aged_data_n):
    data = []
    aged_data = []
    
    for i in range(1, data_n+1):
        tmp_data = pd.read_csv(directory+'/s'+str(i)+'.csv', header=None).values
        data.append(tmp_data)
    
    for i in range(1, aged_data_n+1):
        tmp_data = pd.read_csv(directory+'/s'+str(i)+'_aged.csv', header=None).values
        aged_data.append(tmp_data)
    
    data = np.array(data)
    aged_data = np.array(aged_data)
    
    return data, aged_data

newdata, ageddata = generate_data('fresh_aged_ieice', 50, 2)

In [49]:
"""0の数数える関数"""
def count_zero(data):
    
    tmp = []
    for i in range(data.shape[0]):
        a = (data[i].shape[0] * data[i].shape[1]) - np.count_nonzero(data[i])
        tmp.append(a)
        
    return tmp

In [52]:
"""一次元にする関数"""
def change_flatten(data):
    tmp = []
    for i in range(data.shape[0]):
        tmp.append(data[i].flatten())
        
    tmp = np.array(tmp)
    
    return tmp

In [53]:
"""0を消す関数 flatteしたやつ入れろ"""
def delete_zero(data):
    tmp = []
    for i in range(data.shape[0]):
        tmp2 = copy.deepcopy(data[i])
        tmp.append(tmp2[tmp2 != 0])
        
    tmp = np.array(tmp)
    
    return tmp

In [62]:
"""東西南北残差作り出す関数"""
def EWSN_residual(data):
    tmp_x = [0, 1, 0, -1]
    tmp_y = [-1, 0, 1, 0]

    residual_data = np.zeros_like(data)

    for i in range(data.shape[0]):
        for j in range(data[i].shape[0]):
            for k in range(data[i].shape[1]):
                if data[i, j, k] != 0:
                    data_list = []
                    for l in range(4):
                        next_y = j + tmp_y[l]
                        next_x = k + tmp_x[l]
                        if 0 <= next_y < 148 and 0 <= next_x < 33 and data[i, next_y, next_x] != 0:
                            data_list.append(data[i, next_y, next_x])
    
                    data_mean = mean(data_list)
                    residual_data[i, j, k] = abs(data[i, j, k] - data_mean)

    return residual_data

In [63]:
"""king残差作り出す関数"""
def king_residual(data):
    tmp_x = [-1, 0, 1, 1, 1, 0, -1, -1]
    tmp_y = [-1, -1, -1, 0, 1, 1, 1, 0]
    
    residual_data = np.zeros_like(data)
    
    for i in range(data.shape[0]):
        for j in range(data[i].shape[0]):
            for k in range(data[i].shape[1]):
                if data[i, j, k] != 0:
                    data_list = []
                    for l in range(8):
                        next_y = j + tmp_y[l]
                        next_x = k + tmp_x[l]
                        if 0 <= next_y < 148 and 0 <= next_x < 33 and data[i, next_y, next_x] != 0:
                            data_list.append(data[i, next_y, next_x])
    
                    data_mean = mean(data_list)
                    residual_data[i, j, k] = abs(data[i, j, k] - data_mean)

    return residual_data

In [65]:
"""確率密度関数の可視化"""
def plot_new(data):
    y = 11
    x = 5
    fig,ax = plt.subplots(y,x,figsize=(30,30))
    count = 0
    for i in range(y):
        for j in range(x):
            if count >= 52:
                break
            dummy = sns.distplot(data[count], ax=ax[i, j])
            count += 1
        if count >= 52:
            break
    
    plt.show()

In [67]:
"""リストを繋げる関数"""
def connect(a, b):
    tmp = []
    for i in range(a.shape[0]):
        tmp.append(a[i])
        
    for i in range(b.shape[0]):
        tmp.append(b[i])
    
    tmp = np.array(tmp)
    return tmp

In [72]:
#plot_new(connect(delete_zero(change_flatten(newdata)), delete_zero(change_flatten(ageddata))))

In [75]:
"""QQプロット出力する関数"""
def qq(x):
    stats.probplot(x, dist="norm", plot=plt)  #QQプロット表示
    plt.pause(.01)

In [76]:
"""コルモゴロフスミルノフ検定する関数"""
def kolmogorov_smirnov(x):
    loc, scale = norm.fit(x)
    n = norm(loc=loc, scale=scale)  # create a normal distribution with loc and scale
    p = stats.kstest(x, n.cdf).pvalue  #コルモゴロフスミルノフ検定
    print(p)

In [82]:
"""シャピロウィルク検定する関数"""
def shapiro_wilk(x):
    p=stats.shapiro(x)[1]  #シャピロウィルク検定
    print(p)

In [88]:
a = connect(delete_zero(change_flatten(newdata)), delete_zero(change_flatten(ageddata)))
#a.shape

In [89]:
#for i in range(a.shape[0]):
#    qq(a[i])
#    kolmogorov_smirnov(a[i])
#    shapiro_wilk(a[i])

In [95]:
"""合計値と平均出す関数 一次元混合データが望ましい"""
def summean_residual(data):
    tmpsum = []
    tmpmean = []
    for i in range(data.shape[0]):
        tmpsum.append(data[i].sum())
        tmpmean.append(mean(data[i]))
        
    tmpsum = np.array(tmpsum)
    tmpmean = np.array(tmpmean)
    
    return tmpsum, tmpmean

In [91]:
b, c = sum_residual(a)

In [92]:
print(b, b.shape)
print(c, c.shape)

[419377.7696 421421.0582 445031.9654 466830.7492 474819.9571 470068.975
 459887.8691 463769.8278 462108.0166 462854.7281 480646.5265 437461.6871
 441547.6335 448528.8951 439990.8885 453244.2489 431691.1115 471623.6401
 465244.6996 423192.1911 429656.5336 448291.118  454646.0761 454851.2763
 475216.4795 436424.0284 478118.8372 426163.6995 465357.9904 474120.6986
 457182.0308 461335.1956 439539.5977 453728.391  459602.9678 433687.4494
 445708.6487 419469.0077 468335.4786 463305.3224 458912.8707 450997.5992
 432159.1703 457413.372  457010.3214 441131.5804 448589.5498 444567.4959
 432435.1418 435729.5326 410450.3372 419541.9202] (52,)
[105.79661191 106.31207321 112.26840701 117.76759566 119.7830366
 118.58450429 116.01611229 116.99541569 116.57618986 116.76456309
 121.2529078  110.35864962 111.38941309 113.15057899 110.99669236
 114.34012334 108.90290401 118.97670033 117.36748224 106.75887767
 108.38964016 113.09059485 114.69376289 114.74552883 119.88306748
 110.09687901 120.61524652 107.5

In [93]:
def ten(data):
    x = np.arange(1, 53)
    plt.scatter(x, data)
    plt.title("scatter")
    plt.xlabel("sample number")
    plt.ylabel("residual")
    plt.grid(True)
    plt.show()

In [103]:
#b, c = summean_residual(connect(delete_zero(change_flatten(king_residual(newdata))), delete_zero(change_flatten(king_residual(ageddata)))))
#ten(b)
#ten(c)

In [100]:
def hoteling(data):
    # 標本平均
    mn = mean(data)
    
    # 標本分散
    vari = variance(data)
    
    # 異常度
    anomaly_scores = []
    for x in data:
        anomaly_score = (x - mn)**2 / vari
        anomaly_scores.append(anomaly_score)
    
    # カイ二乗分布による5%水準の閾値
    threshold = stats.chi2.interval(0.95, 1)[1]
    
    # 結果の描画
    num = np.arange(1, 53)
    plt.plot(num, anomaly_scores, "o", color = "b")
    plt.plot([0,53],[threshold, threshold], 'k-', color = "r", ls = "dashed")
    plt.xlabel("Sample number")
    plt.ylabel("Anomaly score")
    plt.show()
    
    for i, j in enumerate(anomaly_scores):
        if j >= threshold:
            print(f'異常index：{i+1}')

In [102]:
#hoteling(b)
#hoteling(c)